1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 젝터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
%pip install --upgrade --quiet docx2txt langchain-community

Note: you may need to restart the kernel to use updated packages.


In [17]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [1]:
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [4]:
len(document_list)

187

In [52]:
#임베딩 OpenAIEmbeddings 사용
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [46]:
from langchain_chroma import Chroma

'''아래 실행하면 chroma라는 폴더가 생성되고 임베딩 된 데이터가 그곳에 생김
chroma 는 인메모리(끄면 사라짐) 라서 persist_directory 의 파일로 저장'''
#database = Chroma.from_documents(documents=document_list, embedding=embedding,collection_name='chroma-tax', persist_directory="./chroma")

database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

In [ ]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
#질문이 있으면 벡터 데이터베이스에 유사도 검색
#retrieved_docs = database.similarity_search(query, k=3)
#retrieved_docs

In [47]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

In [ ]:
'''
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다
- [Context]를 참고해서 사용자의 질문에 답변해 주세요

[Context]
{retrieved_docs}

Question : {query}
"""'
'''

In [ ]:
#ai_message = llm.invoke(prompt)

#ai_message.content

In [25]:
%pip install -u langchain langchainhub —quiet

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\softone\llm-test\.venv\Scripts\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\softone\llm-test\.venv\Scripts\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\softone\llm-test\.venv\Scripts\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\softone\llm-test\.venv\Scripts\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\softone\llm-test\.venv\Scripts\python.exe -m pip install [options] <archive url/path> ...

no such option: -u


In [48]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

c:\Users\softone\llm-test\.venv\lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [34]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [49]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [50]:
#ai_message = qa_chain({"query":query})
ai_message = qa_chain.invoke({"query": query})

In [51]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '죄송하지만, 제공된 정보에는 연봉 5천만원인 직장인의 소득세에 대한 구체적인 내용이 포함되어 있지 않습니다. 따라서 해당 질문에 대한 정확한 답변을 드릴 수 없습니다.'}